In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/altegrad_datachallenge')
from src.inference_manager import TestManager
from src.dataloader import load_data
from src.model import build_model
import torch
import pandas as pd

# D2V Submission

In [2]:
class Config:
    def __init__(self, name_model):
        self.model_dir = "/content/drive/MyDrive/altegrad_datachallenge/"
        self.data_path = "data"
        self.splitpaths = ['d2v.train', 'd2v.valid', 'd2v.test']
        self.seed = 42
        self.author_dim = 128
        self.paper_dim = 256
        self.use_cuda = True
        # Training Parameters
        self.epochs = 60
        self.batch_size = 64
        self.eval_batch_size = 64
        self.validation_freq = 150
        self.training_freq = 50
        # Learning Rate Tunable
        self.lr = 0.0007
        self.patience = 1
        self.factor = 0.8
        # Model
        self.num_heads = 8
        self.num_layers = 2
        self.dropout = 0.4
        self.name_model = name_model

def load_checkpoint(cfg, name):
    model_path = "{}/{}.ckpt".format(cfg.model_dir, name)
    checkpoint = torch.load(model_path, map_location="cuda" if cfg.use_cuda else "cpu")
    return checkpoint

In [3]:
cfg = Config(None)
test_data = load_data(cfg, training=False)

In [4]:
all_predictions = []
split_seeds = ['normal', 0, 5, 10, 15]
names_model_d2v = [f'model_d2v_{split_seed}' for split_seed in split_seeds]

In [5]:
for name_model in names_model_d2v:
    cfg = Config(name_model)
    model = build_model(cfg)
    checkpoint = load_checkpoint(cfg, name_model)
    model.load_state_dict(checkpoint['model_state'])
    test_manager = TestManager(model, cfg)
    predictions, id_author = test_manager.predict(test_data)
    ids_authors = [id_ for ids in id_author for id_ in ids]
    predictions_flatten = [pred.flatten() for pred in predictions]
    predictions_flattened = [y for ys in predictions_flatten for y in ys]
    all_predictions.append(predictions_flattened)

/content/drive/MyDrive/altegrad_datachallenge/src/model.py:41: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(m.weight)
/usr/local/lib/python3.7/dist-packages/torchtext/data/field.py:323: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  var = torch.tensor(arr, dtype=self.dtype, device=device)


In [7]:
# Ensemble predictions with the average strategy
average_predictions = []
for i in range(len(ids_authors)):
    avg_prd = 0
    for j in range(len(names_model)):
        avg_prd += all_predictions[j][i]
    average_pred = round(avg_prd/len(names_model), 5)
    average_predictions.append(average_pred)

In [11]:
submissions = pd.DataFrame({"authorID": ids_authors,"h_index_pred": average_predictions})
submissions.to_csv('/content/drive/MyDrive/altegrad_datachallenge/ensemble_model_d2v.csv', index=False)

# ST Submissions

In [ ]:
class Config:
    def __init__(self, name_model):
        self.model_dir = "/content/drive/MyDrive/altegrad_datachallenge/"
        self.data_path = "data"
        self.splitpaths = ['data.train', 'data.valid', 'data.test']
        self.seed = 42
        self.author_dim = 128
        self.paper_dim = 256
        self.use_cuda = True
        # Training Parameters
        self.epochs = 60
        self.batch_size = 64
        self.eval_batch_size = 64
        self.validation_freq = 150
        self.training_freq = 50
        # Learning Rate Tunable
        self.lr = 0.0007
        self.patience = 1
        self.factor = 0.8
        # Model
        self.num_heads = 8
        self.num_layers = 2
        self.dropout = 0.4
        self.name_model = name_model

def load_checkpoint(cfg, name):
    model_path = "{}/{}.ckpt".format(cfg.model_dir, name)
    checkpoint = torch.load(model_path, map_location="cuda" if cfg.use_cuda else "cpu")
    return checkpoint

In [ ]:
cfg = Config(None)
test_data = load_data(cfg, training=False)

In [ ]:
all_predictions = []
split_seeds = ['normal', 0, 5, 10, 15]
names_model_ST = [f'model_ST_{split_seed}' for split_seed in split_seeds]

In [ ]:
for name_model in names_model_ST:
    cfg = Config(name_model)
    model = build_model(cfg)
    checkpoint = load_checkpoint(cfg, name_model)
    model.load_state_dict(checkpoint['model_state'])
    test_manager = TestManager(model, cfg)
    predictions, id_author = test_manager.predict(test_data)
    ids_authors = [id_ for ids in id_author for id_ in ids]
    predictions_flatten = [pred.flatten() for pred in predictions]
    predictions_flattened = [y for ys in predictions_flatten for y in ys]
    all_predictions.append(predictions_flattened)

In [ ]:
# Ensemble predictions with the average strategy
average_predictions = []
for i in range(len(ids_authors)):
    avg_prd = 0
    for j in range(len(names_model)):
        avg_prd += all_predictions[j][i]
    average_pred = round(avg_prd/len(names_model), 5)
    average_predictions.append(average_pred)

In [ ]:
submissions = pd.DataFrame({"authorID": ids_authors,"h_index_pred": average_predictions})
submissions.to_csv('/content/drive/MyDrive/altegrad_datachallenge/ensemble_model_ST.csv', index=False)

# D2V + ST

In [ ]:
d2v = pd.read_csv('/content/drive/MyDrive/altegrad_datachallenge/ensemble_model_d2v.csv')
st = pd.read_csv('/content/drive/MyDrive/altegrad_datachallenge/ensemble_model_ST.csv')
final_prediction = (d2v.h_index_pred + st.h_index_pred)/2
final_submission = pd.DataFrame({"authorID": d2v.ids_authors,"h_index_pred": final_prediction})
final_submission.to_csv('/content/drive/MyDrive/altegrad_datachallenge/ensemble_model.csv', index=False)